# import

In [10]:
# load all fxns and data
from utils.opconNosepokeFunctions import *
from utils.supplementaryFunctions import *
from utils.plotSettings import *
# from utils.qlfittingFunctions import *
from scipy.optimize import minimize
from scipy.stats import entropy
from scipy.stats import ttest_rel, wilcoxon, mannwhitneyu, ranksums
import statsmodels.api as sm
import statsmodels.formula.api as smf
from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# arms = 4
# current_animals = ['test05022023', 'Blissey', 'Chikorita', 'Darkrai',
#                    'Eevee', 'Goldeen', 'Hoppip', 'Inkay','Jirachi','Kirlia', 'Mesprit']
# boxes = listdir_fullpath("L:/4portProb/")

In [6]:
# sessdf = pd.read_csv('L:/4portProb_processed/sessdf.csv')
import pickle 
with open('L:/4portProb_processed/cleandf.pkl', 'rb') as f:
    sessdf = pickle.load(f)
sessdf.drop(columns = 'Unnamed: 0', inplace = True)
exclude = ['[ 20  20  20 100]', '[0 0 0 0]', '[0]', '[0 0]',
       '[1000   80]', '[30]', '[40]', '[70]']
sessdf = sessdf[~sessdf.rewprobfull.isin(exclude)]
# sessdf = sessdf[~sessdf.duplicated(['trialstart', 'eptime'], keep = False)]

In [7]:
# construct df with rolling windows for -20, +20 trial history (choices), rewards (outcomes)
bin_size = 50
sessdf['sess_bin'] = sessdf.groupby(['animal', 'task'])['session'].transform(lambda x: pd.cut(x, bins=range(0, x.max() + bin_size, bin_size), labels=False, right=False)+1)
mask = (~sessdf.duplicated(subset = ['trialstart', 'eptime'])) & (sessdf.task.isin(['unstr']) & (sessdf.sess_bin>=4))
rolling_df = sessdf[mask].copy(deep = True)
# tempdf = rolling_df.groupby(['animal', 'session']).port.rolling(5, center = True).apply(lambda x: np.mean(np.abs(np.diff(x)))).reset_index()
# rolling_df['distance'] = tempdf.set_index('level_2').port

choice_df = sessdf[mask].copy(deep = True)
hist = 21

for i in range(1, hist):
    
    # shift i tells which port was taken on the prev trial i.e., 1st trial is NaN
    rolling_df['backchoice_t'+str(i)] = sessdf.groupby(['animal', 'session']).port.shift(i)
    
#     shift -i tells which port was taken on the next trial i.e., last trial is NaN
    rolling_df['fwdchoice_t'+str(i)] = sessdf.groupby(['animal', 'session']).port.shift(-i)
    
    # shift i tells outcome on the prev trial i.e., 1st trial is NaN
    rolling_df['backreward_t'+str(i)] = sessdf.groupby(['animal', 'session']).reward.shift(i)

    # shift -i tells outcome on the next trial i.e., last trial is NaN
    rolling_df['fwdreward_t'+str(i)] = sessdf.groupby(['animal', 'session']).reward.shift(-i)

#         # shift i tells which port was taken on th    11e prev trial i.e., 1st trial is NaN
#     rolling_df['backdist_t'+str(i)] = rolling_df.groupby(['animal', 'session']).distance.shift(i)
    
#     # shift -i tells which port was taken on the next trial i.e., last trial is NaN
#     rolling_df['fwddist_t'+str(i)] = rolling_df.groupby(['animal', 'session']).distance.shift(-i)
 
for i in range(1, hist):
        # how much distance travelled due to R at trial 0, and as a control NR - buggy?
    if (i>1):
        rolling_df['backdist_t'+str(i)] = (rolling_df['backchoice_t'+str(i-1)] - rolling_df['backchoice_t'+str(i)]) 
        rolling_df['fwddist_t'+str(i)] = (rolling_df['fwdchoice_t'+str(i-1)] - rolling_df['fwdchoice_t'+str(i)])
    else:
        rolling_df['backdist_t1'] = (rolling_df.port - rolling_df['backchoice_t1'])
        rolling_df['fwddist_t1'] = (rolling_df.port - rolling_df['fwdchoice_t1'])
rolling_df['shift_t0'] = (rolling_df['fwdchoice_t1']==rolling_df['port']).replace({True: 0, False: 1})    
rolling_df = rolling_df.dropna()

#     sessdf.drop(columns = ['choice_t'+str(i), 'choice_t'+str(-i)], inplace = True)

C:\Users\dlab\AppData\Local\Temp\ipykernel_19080\1923324386.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rolling_df['shift_t0'] = (rolling_df['fwdchoice_t1']==rolling_df['port']).replace({True: 0, False: 1})


# Moving Average Entropy 

In [8]:
from scipy.stats import entropy
entropy_df = (choice_df.groupby(['animal', 'session']).port
              .rolling(5, center = True)
              .apply(lambda x: entropy(calc_prob(x), base = 2))
              .reset_index())

In [9]:
choice_df['entropy'] = entropy_df.set_index('level_2').port
for i in range(1, hist):    
    # shift i tells which port was taken on th    11e prev trial i.e., 1st trial is NaN
    choice_df['backentropy_t'+str(i)] = choice_df.groupby(['animal', 'session']).entropy.shift(i)
    
    # shift -i tells which port was taken on the next trial i.e., last trial is NaN
    choice_df['fwdentropy_t'+str(i)] = choice_df.groupby(['animal', 'session']).entropy.shift(-i)
choice_df.dropna(inplace = True)
choice_df['nan'] = 0
cols = ['port', 'reward', 'session', 'task', 'animal']
dist = choice_df[cols+['backentropy_t'+str(i) for i in range(hist-1, 0, -1)]+['nan']+['fwdentropy_t'+str(i) for i in range(1,hist)]]

In [9]:
%matplotlib qt
fig = plt.figure(figsize = (20,10), layout = 'constrained')
plotnum = 1
for animal in dist[~dist.animal.isin(['Yanma', 'Alakazam'])].animal.unique():
    ax = plt.subplot(5, 7, plotnum)
    mask1 = (dist.reward==1) & (dist.animal==animal)
    mask0 = (dist.reward==0) & (dist.animal==animal)
    mat1 = dist[mask1][['backentropy_t'+str(i) for i in range(hist-1, 0, -1)]+['nan']+['fwdentropy_t'+str(i) for i in range(1,hist)]].to_numpy()
    mat0 = dist[mask0][['backentropy_t'+str(i) for i in range(hist-1, 0, -1)]+['nan']+['fwdentropy_t'+str(i) for i in range(1,hist)]].to_numpy()
    var1 = np.var(mat1, axis = 0)
    mea1 = np.mean(mat1, axis=0)
    mea1[hist-3:hist+2], var1[hist-3:hist+2] = np.nan, np.nan
    var0 = np.var(mat0, axis = 0) 
    mea0 = np.mean(mat0, axis=0)
    mea0[hist-3:hist+2], var0[hist-3:hist+2] = np.nan, np.nan
    ax.set_title(animal)
#     ax.plot(np.arange(-hist+1, hist), mea1, 'r', label = "reward")
#     ax.plot(np.arange(-hist+1, hist), mea0, 'b', label = "no reward")
    # plt.fill_between(np.arange(-hist+1, hist), (mea1+stddev1), (mea1-stddev1), color = 'r', alpha = 0.1)
    # plt.fill_between(np.arange(-hist+1, hist), (mea0+stddev0), (mea0-stddev0), color = 'b', alpha = 0.1)
    ax.plot(np.arange(-hist+1, hist), mea0-mea1, 'k', label='Original')
    plotnum+=1
# plt.legend()
# fig.supylabel('Entropy due to single trial outcome (bits)')
fig.supylabel('$\Delta$ entropy due to single trial outcome (bits), No reward $\minus$ Reward', fontsize = 12)
plt.suptitle('Correlated data')
sns.despine()
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_10716\1280115167.py:29: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


# Propensity Score Matching

In [11]:
collist = ['backreward_t'+str(i) for i in range(hist-1, 0, -1)]+['fwdreward_t'+str(i) for i in range(1, hist)]

# make formula for logreg
formula = ''.join([i+' +'for i in collist])
formula = 'reward ~' + formula.rstrip('+')
print(formula)

# define percentiles for binning
from scipy.stats import scoreatpercentile as satp
percentiles = np.linspace(0,1,21)*100
rolling_df.dropna(inplace = True)
rolling_df.prop_score, rolling_df.binnum = 0, 0

# calculate prop score using these columns - for each animal, separately
for animal, group in rolling_df.groupby('animal'):
    # so first, make a logreg
    model = smf.glm(formula = formula, data = group, family = sm.families.Binomial()).fit()
#     print(model.summary())
    
    # make predictions on dataset - this is the prop score (probability of reward given history/future)
    mask = (rolling_df.animal==animal)
    rolling_df.loc[mask, 'prop_score'] = model.predict(group)

for animal, group in rolling_df.groupby('animal'):    
    # bin the scores acc to some logic - use 10 equal prop bins, for eg - assign bin number to group
    mask = (rolling_df.animal==animal)
    binedges = satp(group['prop_score'], percentiles)
#     print(binedges)
    rolling_df.loc[mask, 'binnum'] = np.digitize(rolling_df.loc[mask, 'prop_score'], binedges)
    
# get whether probability corresponds to reward or no reward
# print(np.column_stack((model.model.endog, group.reward.to_numpy())))

# add a column to convert prop score to prediction
rolling_df['pred'] = [1 if x < 0.5 else 0 for x in rolling_df['prop_score']]

# take entropy of trials with equal binnumbers and average? i think
# split before average! into reward and no reward
choice_df['prop_score'] = rolling_df['prop_score']
choice_df['binnum'] = rolling_df['binnum']

collist = ['backentropy_t'+str(i) for i in range(hist-1, 0, -1)]+['nan']+['fwdentropy_t'+str(i) for i in range(1, hist)]

# across animals average in each bin
# remove last bin of both R and NR
lastbin = len(percentiles)
propscored = choice_df.groupby(['animal', 'reward', 'binnum'])[collist].mean().drop(index = lastbin, level = 2).reset_index()

# test whether any of the bins are empty
# print((choice_df.groupby(['reward', 'binnum'])[collist].count()==0).sum())

reward ~backreward_t20 +backreward_t19 +backreward_t18 +backreward_t17 +backreward_t16 +backreward_t15 +backreward_t14 +backreward_t13 +backreward_t12 +backreward_t11 +backreward_t10 +backreward_t9 +backreward_t8 +backreward_t7 +backreward_t6 +backreward_t5 +backreward_t4 +backreward_t3 +backreward_t2 +backreward_t1 +fwdreward_t1 +fwdreward_t2 +fwdreward_t3 +fwdreward_t4 +fwdreward_t5 +fwdreward_t6 +fwdreward_t7 +fwdreward_t8 +fwdreward_t9 +fwdreward_t10 +fwdreward_t11 +fwdreward_t12 +fwdreward_t13 +fwdreward_t14 +fwdreward_t15 +fwdreward_t16 +fwdreward_t17 +fwdreward_t18 +fwdreward_t19 +fwdreward_t20 


In [12]:
mat_dict = {}
fig = plt.figure(figsize = (20,10))
plotnum = 1
for animal, group in propscored.groupby('animal'):
    ax = plt.subplot(5,7, plotnum)
    # no reward - reward per bin
    dif = group[group.reward==0][collist].to_numpy() - group[group.reward==1][collist].to_numpy()

    # now average across bins
    mat = np.mean(dif, axis = 0)
    mat[hist-3:hist+2]= np.nan
    mat_dict[animal] = mat
    
    # plot psmacthed entropy
    ax.plot(np.arange(-hist+1, hist), mat, 'k')
    ax.axvline(0, color = 'k', linestyle = '--')
    ax.set_title(animal)
    ax.set_ylim(0)
    ax.set_xlim(-20, 20)
    plotnum+=1

fig.supylabel('$\Delta$ entropy due to single trial outcome (bits), No reward $\minus$ Reward')
plt.suptitle('Propensity score matched entropy')
sns.despine()
plt.tight_layout()

In [13]:
plt.figure(figsize = (5, 4.5))
# plt.plot(np.arange(-hist+1, hist), mea0-mea1, 'k', label = 'Original')
from scipy.stats import sem
temp=0
for animal, mat in mat_dict.items():
    temp+=mat
plt.plot(np.arange(-hist+1, hist), temp/len(mat_dict), 'xkcd:cornflower', label = 'PS matched')

plt.fill_between(np.arange(-hist+1, hist), (temp/len(mat_dict))-sem(np.array(list(mat_dict.values()))),
                 (temp/len(mat_dict))+sem(np.array(list(mat_dict.values()))),color = 'xkcd:cornflower', alpha = 0.2)
plt.axvline(0, color = 'k', linestyle = '--')
plt.ylabel('$\Delta$ entropy due to single trial outcome, \n No reward $\minus$ Reward', fontsize = 'x-large')
# plt.title('All task structures')
plt.xlabel('Trial history', fontsize = 'x-large')
# plt.legend()
plt.ylim(0, 0.08)
plt.yticks([0.00, 0.02, 0.04, 0.06, 0.08], [0.00, '', '', '', 0.08], fontsize = 'large', color = 'grey')
# plt.yticks(color = 'grey', fontsize = 'large')
plt.xticks(color = 'grey', fontsize = 'large')
sns.despine()
plt.tight_layout()
# plt.savefig('C:/Users/dlab/OneDrive - Indian Institute of Science/Drawings/awp1/prop_score_matched.svg', dpi = 300)

In [100]:
a = np.array([arr[23:] for arr in mat_dict.values()])
def f(x, a, b, c):
    '''exponential fxn: a is amplitude, b is decay rate, c is offset'''
    return a*np.exp(-b*x) + c
from scipy.optimize import curve_fit
popt_str = np.zeros((a.shape[0], 3))
for i in range(a.shape[0]):
    popt_str[i], _ = curve_fit(f, np.arange(a.shape[1]), a[i], nan_policy = 'omit')


# Simulated data 

In [ ]:
################################## simulated q learning
# info
arms = 4
params = [(0.1, 0.1)]
mask = (~sessdf.duplicated(subset = ['trialstart', 'eptime'])) & (~sessdf.task.isin(['dms', 'dls']))
sim_qldf_list = []

# reward prob generator
def fxn_str(mean, arms):
    x = np.linspace(1, arms, arms)
    sig = 1.75/2
    amp = 0.7
    vo = 0.1
    gx = (amp*np.exp(-0.5*((x-mean)**2)/(sig**2)))+vo
    return gx

def fxn_unstr(mean, arms):
    x = np.linspace(1, arms, arms)
    sig = 1.75/2
    amp = 0.7
    vo = 0.1
    gx = (amp*np.exp(-0.5*((x-mean)**2)/(sig**2)))+vo
    gx = np.random.permutation(gx)
    return gx

def rewarding(prob, reward_val):
    temp = reward_val
    rand = np.random.uniform(0, 1)
    return temp if rand <= prob else 0


for paramset in params:  
    alpha, tau = paramset
    
    for (animal, task), dataset in sessdf[mask].groupby(['animal','task']):
        q = np.zeros(arms)
        if 'unstr' in task:
            
            for sessnum, group in dataset.groupby('session'):
                
                rp = fxn_unstr(np.random.choice(np.arange(1,5)), arms)

                for ind, trial in group.iterrows():
                    # softmax prob of choosing actions
                    invtemp=1/tau
                    P = np.exp(invtemp*(q))
                    P = P/ np.sum(P)

                    # which action on this trial
                    a_ind = np.random.multinomial(1, P)
                    a = range(arms)[a_ind.nonzero()[0][0]]

                    # probability of each action on this trial
                    p = P[a]

                    # rewarded?
                    r = rewarding(rp[a], 1)

                    # compute q value
                    q[a] = q[a] + alpha*(r - q[a])
                    
                    # store everything
                    sim_qldf_list.append([animal, task, alpha, tau, sessnum, rp[a], ind, a + 1, r, q[a]])
                    
        else:
            
            for sessnum, group in dataset.groupby('session'):
#                 q = np.zeros(arms)
                p = np.zeros(group.shape[0])
                rp = fxn_str(np.random.choice(np.arange(1,5)), arms)

                for ind, trial in group.iterrows():
                    # softmax prob of choosing actions
                    invtemp=1/tau
                    P = np.exp(invtemp*(q))
                    P = P/ np.sum(P)

                    # which action on this trial
                    a_ind = np.random.multinomial(1, P)
                    a = range(arms)[a_ind.nonzero()[0][0]]

                    # probability of each action on this trial
                    p = P[a]

                    # rewarded?
                    r = rewarding(rp[a], 1)

                    # compute q value
                    q[a] = q[a] + alpha*(r - q[a])
                    
                    # store everything
                    sim_qldf_list.append([animal, task, alpha, tau, sessnum, rp[a], ind, a + 1, r, q[a]])
    
        print('iter done,', paramset, task, animal)
sim_qldf = pd.DataFrame(sim_qldf_list, columns=['animal', 'task', 'alpha',
                                                'tau', 'session', 'rewprob',
                                                'trial#', 'port', 'reward', 'qvalue'])

In [ ]:
# construct df with rolling windows for -20, +20 trial history (choices), rewards (outcomes)
rolling_df_sim = sim_qldf.copy(deep = True)
choice_df_sim = sim_qldf.copy(deep = True)
hist = 21

for i in range(1, hist):
    
    # shift i tells which port was taken on the prev trial i.e., 1st trial is NaN
    rolling_df_sim['backchoice_t'+str(i)] = sessdf.groupby(['animal', 'session']).port.shift(i)
    
#     shift -i tells which port was taken on the next trial i.e., last trial is NaN
    rolling_df_sim['fwdchoice_t'+str(i)] = sessdf.groupby(['animal', 'session']).port.shift(-i)
    
    # shift i tells outcome on the prev trial i.e., 1st trial is NaN
    rolling_df_sim['backreward_t'+str(i)] = sessdf.groupby(['animal', 'session']).reward.shift(i)

    # shift -i tells outcome on the next trial i.e., last trial is NaN
    rolling_df_sim['fwdreward_t'+str(i)] = sessdf.groupby(['animal', 'session']).reward.shift(-i)

#         # shift i tells which port was taken on the prev trial i.e., 1st trial is NaN
#     rolling_df_sim['backdist_t'+str(i)] = rolling_df_sim.groupby(['animal', 'session']).distance.shift(i)
    
#     # shift -i tells which port was taken on the next trial i.e., last trial is NaN
#     rolling_df_sim['fwddist_t'+str(i)] = rolling_df_sim.groupby(['animal', 'session']).distance.shift(-i)
    
        # how much distance travelled due to R at trial 0, and as a control NR - buggy?
    rolling_df_sim['backdist_t'+str(i)] = (rolling_df_sim.port - rolling_df_sim['backchoice_t'+str(i)]) 
    rolling_df_sim['fwddist_t'+str(i)] = (rolling_df_sim.port - rolling_df_sim['fwdchoice_t'+str(i)])
    
#     sessdf.drop(columns = ['choice_t'+str(i), 'choice_t'+str(-i)], inplace = True)


In [ ]:
entropy_df_sim = choice_df_sim.groupby(['animal', 'session']).port.rolling(5, center = True).apply(lambda x: entropy(calc_prob(x), base = 2)).reset_index()

In [ ]:
choice_df_sim['entropy'] = entropy_df_sim.set_index('level_2').port
for i in range(1, hist):    
    # shift i tells which port was taken on th    11e prev trial i.e., 1st trial is NaN
    choice_df_sim['backentropy_t'+str(i)] = choice_df_sim.groupby(['animal', 'session']).entropy.shift(i)
    
    # shift -i tells which port was taken on the next trial i.e., last trial is NaN
    choice_df_sim['fwdentropy_t'+str(i)] = choice_df_sim.groupby(['animal', 'session']).entropy.shift(-i)
choice_df_sim.dropna(inplace = True)
choice_df_sim['nan'] = 0
cols = ['port', 'reward', 'session', 'task', 'animal']
dist = choice_df_sim[cols+['backentropy_t'+str(i) for i in range(hist-1, 0, -1)]+['nan']+['fwdentropy_t'+str(i) for i in range(1,hist)]]

In [ ]:
##################################### PROPENSITY SCORE MATCHING ############################################
collist = ['backreward_t'+str(i) for i in range(hist-1, 0, -1)]+['fwdreward_t'+str(i) for i in range(1, hist)]

# make formula for logreg
formula = ''.join([i+' +'for i in collist])
formula = 'reward ~' + formula.rstrip('+')
print(formula)

# define percentiles for binning
from scipy.stats import scoreatpercentile as satp
percentiles = np.linspace(0,1,11)*100
rolling_df_sim.dropna(inplace = True)
rolling_df_sim.prop_score, rolling_df_sim.binnum = 0, 0

# calculate prop score using these columns - for each animal, separately
for animal, group in rolling_df_sim.groupby('animal'):
    # so first, make a logreg
    model = smf.glm(formula = formula, data = group, family = sm.families.Binomial()).fit()
#     print(model.summary())
    
    # make predictions on dataset - this is the prop score (probability of reward given history/future)
    mask = (rolling_df_sim.animal==animal)
    rolling_df_sim.loc[mask, 'prop_score'] = model.predict(group)

for animal, group in rolling_df_sim.groupby('animal'):    
    # bin the scores acc to some logic - use 10 equal prop bins, for eg - assign bin number to group
    mask = (rolling_df_sim.animal==animal)
    binedges = satp(group['prop_score'], percentiles)
#     print(binedges)
    rolling_df_sim.loc[mask, 'binnum'] = np.digitize(rolling_df_sim.loc[mask, 'prop_score'], binedges)
    
# get whether probability corresponds to reward or no reward
# print(np.column_stack((model.model.endog, group.reward.to_numpy())))

# add a column to convert prop score to prediction
rolling_df_sim['pred'] = [1 if x < 0.5 else 0 for x in rolling_df_sim['prop_score']]

# take entropy of trials with equal binnumbers and average? i think
# split before average! into reward and no reward
choice_df_sim['prop_score'] = rolling_df_sim['prop_score']
choice_df_sim['binnum'] = rolling_df_sim['binnum']

collist = ['backentropy_t'+str(i) for i in range(hist-1, 0, -1)]+['nan']+['fwdentropy_t'+str(i) for i in range(1, hist)]

# across animals average in each bin
# remove last bin of both R and NR
lastbin = len(percentiles)
propscored_sim = choice_df_sim.groupby(['animal', 'reward', 'binnum'])[collist].mean().drop(index = lastbin, level = 2).reset_index()

# test whether any of the bins are empty
# print((choice_df.groupby(['reward', 'binnum'])[collist].count()==0).sum())

In [ ]:
mat_dict_sim = {}
# fig = plt.figure(figsize = (8,8))
plotnum = 1
for animal, group in propscored_sim.groupby('animal'):
#     ax = plt.subplot(4,3, plotnum)
    # no reward - reward per bin
    dif = group[group.reward==0][collist].to_numpy() - group[group.reward==1][collist].to_numpy()

    # now average across bins
    mat = np.mean(dif, axis = 0)
    mat[hist-3:hist+2]= np.nan
    mat_dict_sim[animal] = mat

# fig.supylabel('$\Delta$ entropy due to single trial outcome (bits), No reward $\minus$ Reward')
# plt.suptitle('Propensity score matched entropy')
# sns.despine()
# plt.tight_layout()

In [ ]:
temp =0
temp1 = 0
for animal, mat in mat_dict.items():
    temp+=mat
    temp1 += mat_dict_sim[animal]
plt.plot(np.arange(-hist+1, hist), temp/len(mat_dict), 'r', label = 'real')
plt.plot(np.arange(-hist+1, hist), temp1/len(mat_dict_sim), 'k', label = 'sim')

# plt.fill_between(np.arange(-hist+1, hist), sem())
plt.axvline(0, color = 'k', linestyle = '--')
plt.ylabel('$\Delta$ entropy due to single trial outcome (bits), No reward $\minus$ Reward', wrap = True)
plt.title('All task structures')
plt.legend()
sns.despine()

# Lesion data 

In [14]:
# construct df with rolling windows for -20, +20 trial history (choices), rewards (outcomes)
mask = (~sessdf.duplicated(subset = ['trialstart', 'eptime'])) & (sessdf.task.isin(['dms', 'dls', 'sham']))
rolling_df_dms = sessdf[mask].copy(deep = True)
tempdf = rolling_df_dms.groupby(['animal', 'session']).port.rolling(5, center = True).apply(lambda x: np.mean(np.abs(np.diff(x)))).reset_index()
rolling_df_dms['distance'] = tempdf.set_index('level_2').port

choice_df_dms = sessdf[mask].copy(deep = True)
hist = 21

for i in range(1, hist):
    
    # shift i tells which port was taken on the prev trial i.e., 1st trial is NaN
#     rolling_df_dms['backchoice_t'+str(i)] = sessdf.groupby(['animal', 'session']).port.shift(i)
    
    # shift -i tells which port was taken on the next trial i.e., last trial is NaN
#     rolling_df_dms['fwdchoice_t'+str(i)] = sessdf.groupby(['animal', 'session']).port.shift(-i)
    
    # shift i tells outcome on the prev trial i.e., 1st trial is NaN
    rolling_df_dms['backreward_t'+str(i)] = sessdf[mask].groupby(['animal', 'session']).reward.shift(i)

    # shift -i tells outcome on the next trial i.e., last trial is NaN
    rolling_df_dms['fwdreward_t'+str(i)] = sessdf[mask].groupby(['animal', 'session']).reward.shift(-i)

        # shift i tells which port was taken on th    11e prev trial i.e., 1st trial is NaN
    rolling_df_dms['backdist_t'+str(i)] = rolling_df_dms.groupby(['animal', 'session']).distance.shift(i)
    
    # shift -i tells which port was taken on the next trial i.e., last trial is NaN
    rolling_df_dms['fwddist_t'+str(i)] = rolling_df_dms.groupby(['animal', 'session']).distance.shift(-i)

In [15]:
collist = ['backreward_t'+str(i) for i in range(hist-1, 0, -1)]+['fwdreward_t'+str(i) for i in range(1, hist)]

# make formula for logreg
formula = ''.join([i+' +'for i in collist])
formula = 'reward ~' + formula.rstrip('+')
print(formula)

# define percentiles for binning
from scipy.stats import scoreatpercentile as satp
percentiles = np.linspace(0,1,11)*100
rolling_df_dms.prop_score, rolling_df_dms.binnum = 0, 0

entropy_df_dms = (choice_df_dms.groupby(['animal', 'session'])
                  .port
                  .rolling(5, center = True)
                  .apply(lambda x: entropy(calc_prob(x), base = 2))
                  .reset_index())

choice_df_dms['entropy'] = entropy_df_dms.set_index('level_2').port

for i in range(1, hist):    
    # shift i tells which port was taken on the prev trial i.e., 1st trial is NaN
    choice_df_dms['backentropy_t'+str(i)] = choice_df_dms.groupby(['animal', 'session']).entropy.shift(i)
    
    # shift -i tells which port was taken on the next trial i.e., last trial is NaN
    choice_df_dms['fwdentropy_t'+str(i)] = choice_df_dms.groupby(['animal', 'session']).entropy.shift(-i)
    
choice_df_dms.dropna(inplace = True)
choice_df_dms['nan'] = 0
# cols = ['port', 'reward', 'session', 'task', 'animal']

# calculate prop score using these columns - for each animal, separately
for animal, group in rolling_df_dms.groupby('animal'):
    # so first, make a logreg
    model = smf.glm(formula = formula, data = group, family = sm.families.Binomial()).fit()
#     print(model.summary())
    
    # make predictions on dataset - this is the prop score (probability of reward given history/future)
    mask = (rolling_df_dms.animal==animal)
    rolling_df_dms.loc[mask, 'prop_score'] = model.predict(group)

for animal, group in rolling_df_dms.groupby('animal'):    
#     bin the scores acc to some logic - use 10 equal prop bins, for eg - assign bin number to group
    mask = (rolling_df_dms.animal==animal)
    binedges = satp(group['prop_score'], percentiles)
#     print(binedges)
    rolling_df_dms.loc[mask, 'binnum'] = np.digitize(rolling_df_dms.loc[mask, 'prop_score'], binedges)
    
# get whether probability corresponds to reward or no reward
# print(np.column_stack((model.model.endog, group.reward.to_numpy())))

# add a column to convert prop score to prediction
rolling_df_dms['pred'] = [1 if x < 0.5 else 0 for x in rolling_df_dms['prop_score']]

# take entropy of trials with equal binnumbers and average? i think
# split before average! into reward and no reward
choice_df_dms['prop_score'] = rolling_df_dms['prop_score']
choice_df_dms['binnum'] = rolling_df_dms['binnum']

collist = ['backentropy_t'+str(i) for i in range(hist-1, 0, -1)]+['nan']+['fwdentropy_t'+str(i) for i in range(1, hist)]

# across animals average in each bin
# remove last bin of both R and NR
lastbin = choice_df_dms.binnum.max()
propscored_dms = (choice_df_dms.groupby(['animal', 'reward', 'binnum'])[collist]
                  .mean()
                  .drop(index = lastbin, level = 2)
                  .reset_index())

# test whether any of the bins are empty
# print((choice_df.groupby(['reward', 'binnum'])[collist].count()==0).sum())

reward ~backreward_t20 +backreward_t19 +backreward_t18 +backreward_t17 +backreward_t16 +backreward_t15 +backreward_t14 +backreward_t13 +backreward_t12 +backreward_t11 +backreward_t10 +backreward_t9 +backreward_t8 +backreward_t7 +backreward_t6 +backreward_t5 +backreward_t4 +backreward_t3 +backreward_t2 +backreward_t1 +fwdreward_t1 +fwdreward_t2 +fwdreward_t3 +fwdreward_t4 +fwdreward_t5 +fwdreward_t6 +fwdreward_t7 +fwdreward_t8 +fwdreward_t9 +fwdreward_t10 +fwdreward_t11 +fwdreward_t12 +fwdreward_t13 +fwdreward_t14 +fwdreward_t15 +fwdreward_t16 +fwdreward_t17 +fwdreward_t18 +fwdreward_t19 +fwdreward_t20 


In [16]:
%matplotlib qt
mat_dict_dms = {}
fig = plt.figure(figsize = (20, 10))

plotnum = 1
dls_an = ['Blissey', 'Darkrai', 'Inkay', 'Mesprit', 'Quilava', 'Raltz', 'Zacian', 'Alakazam']
dms_an = ['Hoppip', 'Kirlia', 'Nidorina', 'Togepi', 'Xatu', 'Yanma', 'Giratina', 'Ivysaur', 'Lugia']
sham_an = ['test05022023', 'Jirachi', 'Goldeen', 'Phione', 'Umbreon', 'Vulpix', 'Emolga', 'Haxorus', 'Jigglypuff']
fish = ['test05022023', 'Blissey', 'Chikorita', 'Darkrai', 'Eevee']
for animal, group in propscored_dms.groupby('animal'):
    mask1 = (choice_df_dms.reward==1) & (choice_df_dms.animal==animal)
    mask0 = (choice_df_dms.reward==0) & (choice_df_dms.animal==animal)
    mat1 = choice_df_dms[mask1][['backentropy_t'+str(i) for i in range(hist-1, 0, -1)]+['nan']+['fwdentropy_t'+str(i) for i in range(1,hist)]].to_numpy()
    mat0 = choice_df_dms[mask0][['backentropy_t'+str(i) for i in range(hist-1, 0, -1)]+['nan']+['fwdentropy_t'+str(i) for i in range(1,hist)]].to_numpy()
    var1 = np.var(mat1, axis = 0)
    mea1 = np.mean(mat1, axis=0)
    
    mea1[hist-3:hist+2], var1[hist-3:hist+2] = np.nan, np.nan
    var0 = np.var(mat0, axis = 0) 
    mea0 = np.mean(mat0, axis=0)
    mea0[hist-3:hist+2], var0[hist-3:hist+2] = np.nan, np.nan
    
    ax = plt.subplot(4, 7, plotnum)
    
    # no reward - reward per bin
    dif = group[group.reward==0][collist].to_numpy() - group[group.reward==1][collist].to_numpy()

    # now average across bins
    mat = np.nanmean(dif, axis = 0)
    mat[hist-3:hist+2]= np.nan
    mat_dict_dms[animal] = mat
    mat2 = mat_dict[animal]
    
#     sns.heatmap(dif, ax = ax)
    # plot psmacthed entropy
    
    if animal in dls_an:
        ax.plot(np.arange(1-hist, hist), mat2, 'k', markersize = 7)
        ax.plot(np.arange(1-hist, hist), mat, color = 'xkcd:gold', markersize = 7)
        ax.set_title(animal +', DLS lesion')
        plotnum+=1
    # elif (animal=='Chikorita')| (animal=='Eevee'):
    #     ax.plot(np.arange(1-hist, hist), mat2, 'k', markersize = 7)
    #     ax.plot(np.arange(1-hist, hist), mat, 'xkcd:orange', markersize = 7)
    #     ax.set_title(animal +', DS lesion')
    #     plotnum+=1
    elif animal in dms_an:
        ax.plot(np.arange(1-hist, hist), mat2, 'k', markersize = 7)
        ax.plot(np.arange(1-hist, hist), mat, color='xkcd:red', markersize = 7)
        ax.set_title(animal +', DMS lesion')
        plotnum+=1
    elif animal in sham_an:
        ax.plot(np.arange(1-hist, hist), mat2, 'k', markersize = 7)
        ax.plot(np.arange(1-hist, hist), mat, 'b', markersize = 7)
        ax.set_title(animal +', Sham lesion')
        plotnum+=1
    ax.set_ylim(top = 0.12)
    ax.axvline(0, color = 'k', linestyle = '--')
    ax.set_xlim(-20, 20)
    ax.set_xlabel('Trials')
    

fig.supylabel('$\Delta$ entropy due to single trial outcome (bits), No reward $\minus$ Reward')
plt.suptitle('Propensity score matched distance')
sns.despine()
plt.tight_layout()

In [17]:
temp1 = []
temp = []
%matplotlib qt
cond = 'sham'

# an_list = sessdf[(sessdf.task == cond)].animal.sort_values().unique()
an_list = sham_an
for animal, mat in mat_dict_dms.items():
    
    if animal in an_list:
        temp1.append(mat)
        temp.append(mat_dict[animal])

# calculate mean and sem for all
avg_temp = np.mean(temp, axis = 0)
sem_temp = sem(temp, axis = 0, nan_policy = 'omit')
avg_temp1 = np.mean(temp1, axis = 0)
sem_temp1 = sem(temp1, axis = 0, nan_policy = 'omit')


# plot now!
plt.figure(figsize = (7, 6))
plt.plot(np.arange(-hist+1, hist), avg_temp, 'xkcd:cornflower', label = 'Pre-lesion', lw = 2)
plt.fill_between(np.arange(-hist+1, hist), avg_temp - sem_temp,
                  avg_temp + sem_temp, color = 'xkcd:cornflower', alpha = 0.2)


plt.plot(np.arange(-hist+1, hist), avg_temp1, 'xkcd:slate grey', label = 'Post-lesion', lw = 2)
plt.fill_between(np.arange(-hist+1, hist), avg_temp1 - sem_temp1,
                  avg_temp1 + sem_temp1, color = 'xkcd:slate grey', alpha = 0.2)


plt.axvline(0, color = 'k', linestyle = '--')
plt.ylabel('$\Delta$ entropy due to single trial outcome (bits), \n No reward $\minus$ Reward', fontsize = 'x-large')
plt.xlabel('Trials from conditional', fontsize = 'x-large')
# plt.title('Propensity score matched')
plt.title(f'{cond}, n = {len(an_list)}')
plt.legend()
plt.xticks(color = 'grey', fontsize = 'large')
plt.yticks(color = 'grey', fontsize = 'large')
# plt.ylim(-0.02, 0.07)
sns.despine()
plt.tight_layout()
# plt.savefig('C:/Users/dlab/rishika_sim/plots/20231210/psmatched_entropy_all.png', dpi = 300)

C:\Users\dlab\AppData\Local\Temp\ipykernel_19080\3694288368.py:16: SmallSampleWarning: After omitting NaNs, one or more axis-slices of one or more sample arguments is too small; corresponding elements of returned arrays will be NaN. See documentation for sample size requirements.
  sem_temp = sem(temp, axis = 0, nan_policy = 'omit')
C:\Users\dlab\AppData\Local\Temp\ipykernel_19080\3694288368.py:18: SmallSampleWarning: After omitting NaNs, one or more axis-slices of one or more sample arguments is too small; corresponding elements of returned arrays will be NaN. See documentation for sample size requirements.
  sem_temp1 = sem(temp1, axis = 0, nan_policy = 'omit')


In [158]:
ttest_rel(np.array(temp)[:, 23], np.array(temp1)[:, 23]) 

TtestResult(statistic=np.float64(-0.36146046018922456), pvalue=np.float64(0.7301427320211181), df=np.int64(6))



# Distance Analysis

In [167]:
rolling_df['nan'] = np.nan
cols = ['port', 'reward', 'session', 'task', 'animal']
dist = rolling_df[cols+['backdist_t'+str(i) for i in range(1,hist)]+['nan']+['fwddist_t'+str(i) for i in range(1,hist)]]
mat_dist_rew, mat_dist_unrew = np.zeros(shape = (len(dist.animal.unique()), hist*2-1)), np.zeros(shape = (len(dist.animal.unique()), hist*2-1))

fig = plt.figure(figsize = (20,10), layout = 'constrained')
plotnum = 1
for animal in dist.animal.unique():
    ax = plt.subplot(5, 7, plotnum)
    mask1 = (dist.reward==1) & (dist.animal==animal)
    mask0 = (dist.reward==0) & (dist.animal==animal)
    mat1 = abs(dist[mask1][['backdist_t'+str(i) for i in range(hist-1, 0, -1)]+['nan']+
                           ['fwddist_t'+str(i) for i in range(1,hist)]].to_numpy())
    mat0 = abs(dist[mask0][['backdist_t'+str(i) for i in range(hist-1, 0, -1)]+['nan']+
                           ['fwddist_t'+str(i) for i in range(1,hist)]].to_numpy())
    mea1 = np.mean(mat1, axis=0)
    var1 = np.sum(np.square(mat1 - mea1), axis = 0)/mat1.shape[0]
    mea0 = np.mean(mat0, axis=0)
#     mea1[hist-1], var1[hist-1] = 0,0
    var0 = np.sum(np.square(mat0 - mea0), axis = 0)/mat0.shape[0]  
#     mea0[hist-1], var0[hist-1] = 0,0
    ax.set_title(animal)
#     ax.plot(np.arange(-hist+1, hist), var1, 'r', label = "reward")
#     ax.plot(np.arange(-hist+1, hist), var0, 'b', label = "no reward")
    # plt.fill_between(np.arange(-hist+1, hist), (mea1+stddev1), (mea1-stddev1), color = 'r', alpha = 0.1)
    # plt.fill_between(np.arange(-hist+1, hist), (mea0+stddev0), (mea0-stddev0), color = 'b', alpha = 0.1)
    ax.plot(np.arange(-hist+1, hist), mea0-mea1, '.-k')
    ax.axvline(0, color = 'grey', linestyle = '--')
    mat_dist_rew[plotnum-1] = mea1
    mat_dist_unrew[plotnum-1] = mea0
    plotnum+=1
# plt.legend()
fig.supylabel(r'$\Delta$ distance due to single trial outcome (bits), No reward $\minus$ Reward')
sns.despine()
plt.tight_layout()
# plt.savefig('C:/Users/dlab/rishika_sim/plots/20231210/dist_change_animalwise.png', dpi = 300)

C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\2765576993.py:35: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


In [181]:
mea1 = np.nanmean(mat_dist_rew, axis = 0)
mea0 = np.nanmean(mat_dist_unrew, axis = 0)
sem_sub = sem(mea0-mea1, axis = 0, nan_policy = 'omit')
plt.figure()
plt.plot(np.arange(-hist+1, hist), mea0-mea1, 'k', label = 'original')

plt.fill_between(np.arange(-hist+1, hist), (mea0-mea1)-sem_sub,
                 (mea0-mea1)+sem_sub, color = 'k', alpha = 0.2)
plt.axvline(0, color = 'k', linestyle = '--')
plt.ylabel('$\Delta$ distance due to single trial outcome (bits), No reward $\minus$ Reward', wrap = True, fontsize = 'x-large')
plt.xlabel('Trials from condition', fontsize = 'x-large')
plt.xticks(color = 'grey', fontsize = 'large')
plt.yticks(color = 'grey', fontsize = 'large')
# plt.legend()
sns.despine()
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\881873552.py:1: RuntimeWarning: Mean of empty slice
  mea1 = np.nanmean(mat_dist_rew, axis = 0)
C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\881873552.py:2: RuntimeWarning: Mean of empty slice
  mea0 = np.nanmean(mat_dist_unrew, axis = 0)


In [168]:
########################################### PROPENSITY SCORE MATCHING ###############################################
collist = ['backreward_t'+str(i) for i in range(hist-1, 0, -1)]+['fwdreward_t'+str(i) for i in range(1, hist)]

# make formula for logreg
formula = ''.join([i+' +'for i in collist])
formula = 'reward ~' + formula.rstrip('+')
print(formula)

# define percentiles for binning
from scipy.stats import scoreatpercentile as satp
percentiles = np.linspace(0, 1, 11)*100
rolling_df.prop_score, rolling_df.binnum = 0, 0

# calculate prop score using these columns - for each animal, separately
for animal, group in rolling_df.groupby('animal'):
    # so first, make a logreg
    model = smf.glm(formula = formula, data = group, family = sm.families.Binomial()).fit()
#     print(model.summary())
    
    # make predictions on dataset - this is the prop score (probability of reward given history/future)
    mask = (rolling_df.animal==animal)
    rolling_df.loc[mask, 'prop_score'] = model.predict(group)

for animal, group in rolling_df.groupby('animal'):    
    # bin the scores acc to some logic - use 10 equal prop bins, for eg - assign bin number to group
    mask = (rolling_df.animal==animal)
    binedges = satp(group['prop_score'], percentiles)
#     print(binedges)
    rolling_df.loc[mask, 'binnum'] = np.digitize(rolling_df.loc[mask, 'prop_score'], binedges)
    
# get whether probability corresponds to reward or no reward
# print(np.column_stack((model.model.endog, group.reward.to_numpy())))

# add a column to convert prop score to prediction
rolling_df['pred'] = [1 if x < 0.5 else 0 for x in rolling_df['prop_score']]

# take entropy of trials with equal binnumbers and average? i think
# split before average! into reward and no reward

collist = ['backdist_t'+str(i) for i in range(hist-1, 0, -1)]+['nan']+['fwddist_t'+str(i) for i in range(1, hist)]

# across animals average in each bin
# remove last bin of both R and NR
lastbin = len(percentiles)
rolling_df[collist] = rolling_df[collist].abs()
propscored = rolling_df.groupby(['animal', 'reward', 'binnum'])[collist].mean().drop(index = lastbin, level = 2).reset_index()

# test whether any of the bins are empty
# print((choice_df.groupby(['reward', 'binnum'])[collist].count()==0).sum())

reward ~backreward_t20 +backreward_t19 +backreward_t18 +backreward_t17 +backreward_t16 +backreward_t15 +backreward_t14 +backreward_t13 +backreward_t12 +backreward_t11 +backreward_t10 +backreward_t9 +backreward_t8 +backreward_t7 +backreward_t6 +backreward_t5 +backreward_t4 +backreward_t3 +backreward_t2 +backreward_t1 +fwdreward_t1 +fwdreward_t2 +fwdreward_t3 +fwdreward_t4 +fwdreward_t5 +fwdreward_t6 +fwdreward_t7 +fwdreward_t8 +fwdreward_t9 +fwdreward_t10 +fwdreward_t11 +fwdreward_t12 +fwdreward_t13 +fwdreward_t14 +fwdreward_t15 +fwdreward_t16 +fwdreward_t17 +fwdreward_t18 +fwdreward_t19 +fwdreward_t20 


In [ ]:
mat_dist_ps = {}
fig = plt.figure(figsize = (20,10))
plotnum = 1
for animal, group in propscored.groupby('animal'):
    ax = plt.subplot(5, 7, plotnum)
    # no reward - reward per bin
    dif = abs(group[group.reward==0][collist].to_numpy()) - abs(group[group.reward==1][collist].to_numpy())

    # now average across bins
    mat = np.nanmean(dif, axis = 0)
    mat_dist_ps[animal] = mat
    
    # plot psmacthed entropy
    ax.plot(np.arange(-hist+1, hist), mat, 'k.-')
    ax.axvline(0, color = 'k', linestyle = '--')
    ax.set_title(animal)
    ax.set_ylim(0, 0.2)
    ax.set_xlim(-20, 20)
    plotnum+=1

fig.supylabel('$\Delta$ distance due to single trial outcome (bits), No reward $\minus$ Reward')
plt.suptitle('Propensity score matched distance')
sns.despine()
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\2289018050.py:10: RuntimeWarning: Mean of empty slice
  mat = np.nanmean(dif, axis = 0)
C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\2289018050.py:10: RuntimeWarning: Mean of empty slice
  mat = np.nanmean(dif, axis = 0)
C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\2289018050.py:10: RuntimeWarning: Mean of empty slice
  mat = np.nanmean(dif, axis = 0)
C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\2289018050.py:10: RuntimeWarning: Mean of empty slice
  mat = np.nanmean(dif, axis = 0)
C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\2289018050.py:10: RuntimeWarning: Mean of empty slice
  mat = np.nanmean(dif, axis = 0)
C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\2289018050.py:10: RuntimeWarning: Mean of empty slice
  mat = np.nanmean(dif, axis = 0)
C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\2289018050.py:10: RuntimeWarning: Mean of empty slice
  mat = np.nanmean(dif, axis = 0)
C:\Users\dlab\AppData\Local\Temp\ipykerne

In [188]:
temp = []
%matplotlib qt

for animal, mat in mat_dist_ps.items():
    temp.append(mat)

# calculate mean and sem for all
avg_temp = np.mean(temp, axis = 0)
sem_temp = sem(temp, axis = 0, nan_policy = 'omit')

# plot now!
plt.figure()
plt.plot(np.arange(-hist+1, hist), mea0 - mea1, 'k.-', label = 'original')
plt.plot(np.arange(-hist+1, hist), avg_temp, 'xkcd:cornflower', marker = '.', label = 'PS matched')
plt.fill_between(np.arange(-hist+1, hist), avg_temp - sem_temp,
                  avg_temp + sem_temp, color = 'xkcd:cornflower', alpha = 0.2)


plt.axvline(0, color = 'k', linestyle = '--', alpha = 0.5)
plt.ylabel('$\Delta$ distance due to single trial outcome, \n No reward $\minus$ Reward', fontsize = 'x-large')
plt.xlabel('Trials from condition', fontsize = 'x-large')
# plt.title('Propensity score matched')

plt.legend()
plt.xticks(color = 'grey', fontsize = 'large')
plt.yticks(color = 'grey', fontsize = 'large')
# plt.ylim(-0.02, 0.07)
sns.despine()
plt.tight_layout()

## lesioned data

In [ ]:
# construct df with rolling windows for -20, +20 trial history (choices), rewards (outcomes)
mask = (~sessdf.duplicated(subset = ['trialstart', 'eptime'])) & (sessdf.task.isin(['dms', 'dls', 'sham']))
rolling_df_dms = sessdf[mask].copy(deep = True)

hist = 21

for i in range(1, hist):
    
    # shift i tells which port was taken on the prev trial i.e., 1st trial is NaN
    rolling_df_dms['backchoice_t'+str(i)] = sessdf.groupby(['animal', 'session']).port.shift(i)
    
#     shift -i tells which port was taken on the next trial i.e., last trial is NaN
    rolling_df_dms['fwdchoice_t'+str(i)] = sessdf.groupby(['animal', 'session']).port.shift(-i)
    
    # shift i tells outcome on the prev trial i.e., 1st trial is NaN
    rolling_df_dms['backreward_t'+str(i)] = sessdf.groupby(['animal', 'session']).reward.shift(i)

    # shift -i tells outcome on the next trial i.e., last trial is NaN
    rolling_df_dms['fwdreward_t'+str(i)] = sessdf.groupby(['animal', 'session']).reward.shift(-i)

for i in range(1, hist):
        # how much distance travelled due to R at trial 0, and as a control NR - buggy?
    if (i)>1:
        rolling_df_dms['backdist_t'+str(i)] = (rolling_df_dms['backchoice_t'+str(i-1)] - rolling_df_dms['backchoice_t'+str(i)]) 
        rolling_df_dms['fwddist_t'+str(i)] = (rolling_df_dms['fwdchoice_t'+str(i-1)] - rolling_df_dms['fwdchoice_t'+str(i)])
    rolling_df_dms['backdist_t1'] = (rolling_df_dms.port - rolling_df_dms['backchoice_t1'])
    rolling_df_dms['fwddist_t1'] = (rolling_df_dms.port - rolling_df_dms['fwdchoice_t1'])
    
rolling_df_dms['shift_t0'] = (rolling_df_dms['fwdchoice_t1']==rolling_df_dms['port']).replace({True: 0, False: 1}) 

In [ ]:
# specify cols for ps matching
collist = ['backreward_t'+str(i) for i in range(hist-1, 0, -1)]+['fwdreward_t'+str(i) for i in range(1, hist)]

# make formula for logreg
formula = ''.join([i+' +'for i in collist])
formula = 'reward ~' + formula.rstrip('+')
print(formula)

# define percentiles for binning
from scipy.stats import scoreatpercentile as satp
percentiles = np.linspace(0,1,21)*100
rolling_df_dms.prop_score, rolling_df_dms.binnum = 0, 0


# calculate prop score using these columns - for each animal, separately
for animal, group in rolling_df_dms.groupby('animal'):
    # so first, make a logreg
    model = smf.glm(formula = formula, data = group, family = sm.families.Binomial()).fit()
#     print(model.summary())
    
    # make predictions on dataset - this is the prop score (probability of reward given history/future)
    mask = (rolling_df_dms.animal==animal)
    rolling_df_dms.loc[mask, 'prop_score'] = model.predict(group)

for animal, group in rolling_df_dms.groupby('animal'):    
#     bin the scores acc to some logic - use 10 equal prop bins, for eg - assign bin number to group
    mask = (rolling_df_dms.animal==animal)
    binedges = satp(group['prop_score'], percentiles)
#     print(binedges)
    rolling_df_dms.loc[mask, 'binnum'] = np.digitize(rolling_df_dms.loc[mask, 'prop_score'], binedges)
    
# get whether probability corresponds to reward or no reward
# print(np.column_stack((model.model.endog, group.reward.to_numpy())))

# add a column to convert prop score to prediction
rolling_df_dms['pred'] = [1 if x < 0.5 else 0 for x in rolling_df_dms['prop_score']]

# take entropy of trials with equal binnumbers and average? i think
# split before average! into reward and no reward
collist = ['backdist_t'+str(i) for i in range(hist-1, 0, -1)]+['fwddist_t'+str(i) for i in range(1, hist)]

# across animals average in each bin
# remove last bin of both R and NR
lastbin = len(percentiles)
rolling_df_dms[collist] = rolling_df_dms[collist].abs()
propscored_dms = rolling_df_dms.groupby(['animal', 'reward', 'binnum'])[collist].mean().drop(index = lastbin, level = 2).reset_index()

# test whether any of the bins are empty
# print((choice_df.groupby(['reward', 'binnum'])[collist].count()==0).sum())

reward ~backreward_t20 +backreward_t19 +backreward_t18 +backreward_t17 +backreward_t16 +backreward_t15 +backreward_t14 +backreward_t13 +backreward_t12 +backreward_t11 +backreward_t10 +backreward_t9 +backreward_t8 +backreward_t7 +backreward_t6 +backreward_t5 +backreward_t4 +backreward_t3 +backreward_t2 +backreward_t1 +fwdreward_t1 +fwdreward_t2 +fwdreward_t3 +fwdreward_t4 +fwdreward_t5 +fwdreward_t6 +fwdreward_t7 +fwdreward_t8 +fwdreward_t9 +fwdreward_t10 +fwdreward_t11 +fwdreward_t12 +fwdreward_t13 +fwdreward_t14 +fwdreward_t15 +fwdreward_t16 +fwdreward_t17 +fwdreward_t18 +fwdreward_t19 +fwdreward_t20 


In [103]:
# %matplotlib qt
mat_dist_ps_les = {}
fig = plt.figure(figsize = (20,10))
plotnum = 1
for animal, group in propscored_dms.groupby('animal'):
    ax = plt.subplot(4, 7, plotnum)
    # no reward - reward per bin
    dif = abs(group[group.reward==0][collist].to_numpy()) - abs(group[group.reward==1][collist].to_numpy())

    # now average across bins
    mat = np.mean(dif, axis = 0)
    mat[hist-1]= 0
    mat_dist_ps_les[animal] = mat
    mat2 =mat_dist_ps[animal]
    
#     sns.heatmap(dif, ax = ax)
    # plot psmacthed entropy
    ax.axvline(0, color = 'k', linestyle = '--')
    if (animal=='Blissey')| (animal=='Darkrai')|(animal=='Inkay')|(animal =='Mesprit'):
        ax.plot(np.arange(1-hist, hist-1)+0.5, mat2, 'k', markersize = 7)
        ax.plot(np.arange(1-hist, hist-1)+0.5, mat,color = 'xkcd:gold', markersize = 7)
        ax.set_title(animal +', DLS lesion')
        plotnum+=1
    # elif (animal=='Chikorita')| (animal=='Eevee'):
    #     ax.plot(np.arange(1-hist, hist-1)+0.5, mat2, 'k', markersize = 7)
    #     ax.plot(np.arange(1-hist, hist-1)+0.5, mat, 'xkcd:orange', markersize = 7)
    #     ax.set_title(animal +', DS lesion')
    #     plotnum+=1
    # elif  (animal =='Kirlia')|(animal =='Hoppip')|(animal =='Nidorina'):
    #     ax.plot(np.arange(1-hist, hist-1)+0.5, mat2, 'k', markersize = 7)
    #     ax.plot(np.arange(1-hist, hist-1)+0.5, mat, color='xkcd:red', markersize = 7)
    #     ax.set_title(animal +', DMS lesion')
    #     plotnum+=1
    # elif (animal=='test05022023')|(animal == 'Jirachi')|(animal == 'Goldeen'):
    #     ax.plot(np.arange(1-hist, hist-1)+0.5, mat2, 'k', markersize = 7)
    #     ax.plot(np.arange(1-hist, hist-1)+0.5, mat, 'b', markersize = 7)
    #     ax.set_title(animal +', Sham lesion')
        plotnum+=1
#     ax.set_ylim(0)
    ax.set_xlim(-20, 20)
    ax.set_xlabel('Trials')
    

fig.supylabel('$\Delta$ distance due to single trial outcome (bits), No reward $\minus$ Reward')
plt.suptitle('Propensity score matched distance')
sns.despine()
plt.tight_layout()
# plt.savefig('C:/Users/dlab/rishika_sim/plots/20231210/psmatched_distance_les.png', dpi = 300)

In [109]:
temp1 = []
temp = []
%matplotlib qt
cond = 'dms'

an_list = sessdf[(sessdf.task == cond)].animal.sort_values().unique()
for animal, mat in mat_dist_ps_les.items():
    
    if animal in an_list:
        temp1.append(mat)
        temp.append(mat_dist_ps[animal])

# calculate mean and sem for all
avg_temp = np.mean(temp, axis = 0)
sem_temp = sem(temp, axis = 0, nan_policy = 'omit')
avg_temp1 = np.mean(temp1, axis = 0)
sem_temp1 = sem(temp1, axis = 0, nan_policy = 'omit')


# plot now!
plt.figure(figsize = (7, 6))
plt.plot(np.arange(1-hist, hist-1), avg_temp, 'xkcd:azure', label = 'Pre-lesion')
plt.fill_between(np.arange(1-hist, hist-1), avg_temp - sem_temp,
                  avg_temp + sem_temp, color = 'xkcd:azure', alpha = 0.2)


plt.plot(np.arange(1-hist, hist-1), avg_temp1, 'xkcd:slate grey', label = 'Post-lesion')
plt.fill_between(np.arange(1-hist, hist-1), avg_temp1 - sem_temp1,
                  avg_temp1 + sem_temp1, color = 'xkcd:slate grey', alpha = 0.2)


plt.axvline(0, color = 'k', linestyle = '--')
plt.ylabel('$\Delta$ distance due to single trial outcome (bits),\nNo reward $\minus$ Reward', fontsize = 'x-large', wrap = True)
plt.xlabel('Trials from conditional', fontsize = 'x-large')
# plt.title('Propensity score matched')

plt.legend()
plt.xticks(color = 'grey', fontsize = 'large')
plt.yticks(color = 'grey', fontsize = 'large')
# plt.ylim(-0.02, 0.07)
sns.despine()
plt.tight_layout()
# plt.savefig('C:/Users/dlab/rishika_sim/plots/20231210/psmatched_entropy_all.png', dpi = 300)

# Rew-prob matched entropy 

In [219]:
def calc_shifted_entropy(choice_df): 
    results = []

    # Process for each animal and offset in a single loop
    for n in range(-20, 21):
        if n in [-2, -1, 0, 1, 2]:
            continue

        choice_df['entropy_shifted'] = (
            choice_df.groupby(['animal', 'session'])['entropy'].shift(-n)
        )

        ported = (
            choice_df
            .groupby(['animal', 'session', 'rewprob', 'reward'], as_index=False)['entropy_shifted']
            .mean()
        )

        grouped = ported.groupby(['animal', 'rewprob', 'reward'], as_index=False)['entropy_shifted'].mean()
        grouped['offset'] = n

        # Append to results
        results.append(grouped)

    # Combine results
    final_df = pd.concat(results, ignore_index=True)
    return final_df
final_df = calc_shifted_entropy(choice_df)

In [269]:
rew_an_dict, unrew_an_dict, diff_dict = {}, {}, {}
# Add NaNs in the middle
nan_series = pd.Series([np.nan] * 5, index=range(-2, 3))

for an in final_df.animal.unique():
    # calc for each animal, for each rp
    for rewprob in [10, 15, 46, 80]:
        unrew = (final_df[(final_df['rewprob'] == rewprob) & (final_df['reward'] == 0) & (final_df['animal']==an)]
            .set_index('offset')['entropy_shifted'])
        rew = (final_df[(final_df['rewprob'] == rewprob) & (final_df['reward'] == 1) & (final_df['animal']==an)]
            .set_index('offset')['entropy_shifted'])
    
        rew_an_dict[(an, rewprob)] = pd.concat([rew.iloc[:18], nan_series, rew.iloc[18:]])
        unrew_an_dict[(an, rewprob)] = pd.concat([unrew.iloc[:18], nan_series, unrew.iloc[18:]])
        diff_dict[(an, rewprob)] = unrew_an_dict[(an, rewprob)] - rew_an_dict[(an, rewprob)]
    # if diff_dict[(an, rewprob)].isna().sum() > 0:
    #     print(an, rewprob)
# data = pd.DataFrame(diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)]

In [228]:
colors = sns.color_palette("crest", as_cmap=True)(np.linspace(0, 1, 4))
mean_ = pd.DataFrame(diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)].mean().to_numpy()
sem_ = pd.DataFrame(diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)].sem().to_numpy()
# plt.plot(np.arange(-hist+1, hist), mea0-mea1, 'k', label = 'Original')
# plt.plot(np.arange(-hist+1, hist), temp/len(mat_dict), 'xkcd:coral', label = 'PS matched')

# plt.fill_between(np.arange(-hist+1, hist), (temp/len(mat_dict))-sem(np.array(list(mat_dict.values()))),
#                  (temp/len(mat_dict))+sem(np.array(list(mat_dict.values()))),color = 'xkcd:coral', alpha = 0.2)

for ind, rewprob in enumerate([10, 15, 46, 80]):
    plt.plot(np.arange(-hist+1, hist), mean_[ind], '-', color = colors[ind], label = rewprob)
    plt.fill_between(
        np.arange(-hist+1, hist), mean_[ind] - sem_[ind], mean_[ind] + sem_[ind],
        color=colors[ind], alpha=0.2
    )



plt.axvline(0, color = 'k', linestyle = '--')
plt.ylabel('$\Delta$ entropy due to single trial outcome, \n No reward $\minus$ Reward', fontsize = 'x-large')
# plt.title('All task structures')
plt.xlabel('Trial history', fontsize = 'x-large')
plt.legend()
# plt.ylim(0, 0.08)
# plt.yticks([0.00, 0.02, 0.04, 0.06, 0.08], [0.00, '', '', '', 0.08], fontsize = 'large', color = 'grey')
plt.yticks(color = 'grey', fontsize = 'large')
plt.xticks(color = 'grey', fontsize = 'large')
sns.despine()
plt.tight_layout()

## lesioned data

In [ ]:
final_df_dls = calc_shifted_entropy(choice_df_dms[choice_df_dms.task=='dls'])
final_df_dms = calc_shifted_entropy(choice_df_dms[choice_df_dms.task=='dms'])
final_df_sham = calc_shifted_entropy(choice_df_dms[choice_df_dms.task=='sham'])

C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\3764907387.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choice_df['entropy_shifted'] = (
C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\3764907387.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choice_df['entropy_shifted'] = (
C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\3764907387.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [245]:
# choose a condition
cond = 'sham'

# df based on condition
cond_df = {'dms': final_df_dms,
          'dls': final_df_dls,
          'sham': final_df_sham}[cond]

pre_rew_an_dict, pre_unrew_an_dict, pre_diff_dict = {}, {}, {}
post_rew_an_dict, post_unrew_an_dict, post_diff_dict = {}, {}, {}

# Add NaNs in the middle
nan_series = pd.Series([np.nan] * 5, index=range(-2, 3))
for an in cond_df.animal.unique():
    # calc for each animal, for each rp
    for rewprob in [10, 15, 46, 80]:
        unrew = (final_df[(final_df['rewprob'] == rewprob) & (final_df['reward'] == 0) & (final_df['animal']==an)]
            .set_index('offset')['entropy_shifted'])
        rew = (final_df[(final_df['rewprob'] == rewprob) & (final_df['reward'] == 1) & (final_df['animal']==an)]
            .set_index('offset')['entropy_shifted'])
    
        pre_rew_an_dict[(an, rewprob)] = pd.concat([rew.iloc[:18], nan_series, rew.iloc[18:]])
        pre_unrew_an_dict[(an, rewprob)] = pd.concat([unrew.iloc[:18], nan_series, unrew.iloc[18:]])
        pre_diff_dict[(an, rewprob)] = pre_unrew_an_dict[(an, rewprob)] - pre_rew_an_dict[(an, rewprob)]

        unrew = (cond_df[(cond_df['rewprob'] == rewprob) & (cond_df['reward'] == 0) & (cond_df['animal']==an)]
            .set_index('offset')['entropy_shifted'])
        rew = (cond_df[(cond_df['rewprob'] == rewprob) & (cond_df['reward'] == 1) & (cond_df['animal']==an)]
            .set_index('offset')['entropy_shifted'])
    
        post_rew_an_dict[(an, rewprob)] = pd.concat([rew.iloc[:18], nan_series, rew.iloc[18:]])
        post_unrew_an_dict[(an, rewprob)] = pd.concat([unrew.iloc[:18], nan_series, unrew.iloc[18:]])
        post_diff_dict[(an, rewprob)] = post_unrew_an_dict[(an, rewprob)] - post_rew_an_dict[(an, rewprob)]

colors = sns.color_palette("crest", as_cmap=True)(np.linspace(0, 1, 4))
pre_mean = pd.DataFrame(pre_diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)].mean().to_numpy()
pre_sem = pd.DataFrame(pre_diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)].sem().to_numpy()
mean_ = pd.DataFrame(post_diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)].mean().to_numpy()
sem_ = pd.DataFrame(post_diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)].sem().to_numpy()
# plt.plot(np.arange(-hist+1, hist), mea0-mea1, 'k', label = 'Original')
# plt.plot(np.arange(-hist+1, hist), temp/len(mat_dict), 'xkcd:coral', label = 'PS matched')

# plt.fill_between(np.arange(-hist+1, hist), (temp/len(mat_dict))-sem(np.array(list(mat_dict.values()))),
#                  (temp/len(mat_dict))+sem(np.array(list(mat_dict.values()))),color = 'xkcd:coral', alpha = 0.2)
fig = plt.figure(figsize=(15, 4))
for ind, rewprob in enumerate([10, 15, 46, 80]):
    ax = plt.subplot(1, 4, ind+1)
    ax.plot(np.arange(-hist+1, hist), pre_mean[ind], '-', color = colors[ind], label = rewprob)
    ax.fill_between(
        np.arange(-hist+1, hist), pre_mean[ind] - pre_sem[ind], pre_mean[ind] + pre_sem[ind],
        color=colors[ind], alpha=0.6
    )
    ax.plot(np.arange(-hist+1, hist), mean_[ind], '-', color = colors[ind], label = rewprob, alpha = 0.5)
    ax.fill_between(
        np.arange(-hist+1, hist), mean_[ind] - sem_[ind], mean_[ind] + sem_[ind],
        color=colors[ind], alpha=0.2
    )
    ax.axvline(0, color = 'k', linestyle = '--', alpha = 0.5)
    ax.set_ylim(-0.2, 0.2)
    ax.set_title(f'{rewprob}% reward', fontsize = 'large')
    plt.yticks(color = 'grey', fontsize = 'large')
    plt.xticks(color = 'grey', fontsize = 'large')
    


# fig.supylabel('$\Delta$ entropy due to single trial outcome, \n No reward $\minus$ Reward', fontsize = 'x-large')
plt.suptitle(f'{cond}, n = {len(cond_df.animal.unique())}', fontsize = 'x-large')
# plt.xlabel('Trial history', fontsize = 'x-large')
# plt.legend()
# plt.ylim(0, 0.08)
# plt.yticks([0.00, 0.02, 0.04, 0.06, 0.08], [0.00, '', '', '', 0.08], fontsize = 'large', color = 'grey')

sns.despine()
plt.tight_layout()

# Rew-prob matched distance

In [247]:
choice_df['choice_t1'] = choice_df.groupby(['animal','session']).port.shift(-1)
choice_df.loc[(choice_df.choice_t1 == 0), 'choice_t1'] = choice_df.loc[(choice_df.choice_t1 == 0), 'port']
choice_df['dist'] = abs(choice_df['choice_t1']-choice_df['port'])

In [248]:
def calc_shifted_distance(choice_df):
    results = []

    # Process for each animal and offset in a single loop
    for n in range(-20, 21):
        if n in [0]:
            continue
        
        choice_df['dist_shifted'] = (
            choice_df.groupby(['animal', 'session'])['dist'].shift(-n)
        )

        ported = (
            choice_df
            .groupby(['animal', 'session', 'rewprob', 'reward'], as_index=False)['dist_shifted']
            .mean()
        )

        grouped = ported.groupby(['animal', 'rewprob', 'reward'], as_index=False)['dist_shifted'].mean()
        grouped['offset'] = n

        # Append to results
        results.append(grouped)
        

    # Combine results
    final_df = pd.concat(results, ignore_index=True)
    return final_df
final_df = calc_shifted_distance(choice_df)

In [116]:
rew_an_dict, unrew_an_dict, diff_dict = {}, {}, {}
nan_series = pd.Series([np.nan], index=[0])

for an in final_df.animal.unique():
    # calc for each animal, for each rp
    for rewprob in [10, 15, 46, 80]:
        unrew = (final_df[(final_df['rewprob'] == rewprob) & (final_df['reward'] == 0) & (final_df['animal']==an)]
            .set_index('offset')['dist_shifted'])
        rew = (final_df[(final_df['rewprob'] == rewprob) & (final_df['reward'] == 1) & (final_df['animal']==an)]
            .set_index('offset')['dist_shifted'])
    
        rew_an_dict[(an, rewprob)] = pd.concat([rew.iloc[:20], nan_series, rew.iloc[20:]])
        unrew_an_dict[(an, rewprob)] = pd.concat([unrew.iloc[:20], nan_series, unrew.iloc[20:]])
        diff_dict[(an, rewprob)] = unrew_an_dict[(an, rewprob)] - rew_an_dict[(an, rewprob)]
    # if diff_dict[(an, rewprob)].isna().sum() > 0:
    #     print(an, rewprob)
# data = pd.DataFrame(diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)]

In [ ]:
colors = sns.color_palette("crest", as_cmap=True)(np.linspace(0, 1, 4))
mean_ = pd.DataFrame(diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)].mean().to_numpy()
sem_ = pd.DataFrame(diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)].sem().to_numpy()
# plt.plot(np.arange(-hist+1, hist), mea0-mea1, '.-k', label = 'Original')
# plt.plot(np.arange(-hist+1, hist), avg_temp, 'xkcd:coral', marker = '.', label = 'PS matched')

# plt.fill_between(np.arange(-hist+1, hist), avg_temp - sem_temp,
#                   avg_temp + sem_temp, color = 'xkcd:coral', alpha = 0.2)
plt.figure()

plt.axvline(0, color = 'k', linestyle = '--', alpha = 0.5)

for ind, rewprob in enumerate([10, 15, 46, 80]):
    plt.plot(np.arange(-hist+1, hist), mean_[ind], '.-', color = colors[ind], label = f'Rew. {rewprob}%')
    plt.fill_between(
        np.arange(-hist+1, hist), mean_[ind] - sem_[ind], mean_[ind] + sem_[ind],
        color=colors[ind], alpha=0.2
    )

plt.ylabel('$\Delta$ distance due to single trial outcome, \n No reward $\minus$ Reward', fontsize = 'x-large')
plt.xlabel('Trial history', fontsize = 'x-large')
plt.legend()
# plt.ylim(0, 0.08)
# plt.yticks([0.00, 0.02, 0.04, 0.06, 0.08], [0.00, '', '', '', 0.08], fontsize = 'large', color = 'grey')
plt.yticks(color = 'grey', fontsize = 'large')
plt.xticks(color = 'grey', fontsize = 'large')
sns.despine()
plt.tight_layout()

## lesioned data

In [250]:
choice_df_dms['choice_t1'] = choice_df_dms.groupby(['animal','session']).port.shift(-1)
choice_df_dms.loc[(choice_df_dms.choice_t1 == 0), 'choice_t1'] = choice_df_dms.loc[(choice_df_dms.choice_t1 == 0), 'port']
choice_df_dms['dist'] = abs(choice_df_dms['choice_t1']-choice_df_dms['port'])
final_df_dls = calc_shifted_distance(choice_df_dms[choice_df_dms.task=='dls'])
final_df_dms = calc_shifted_distance(choice_df_dms[choice_df_dms.task=='dms'])
final_df_sham = calc_shifted_distance(choice_df_dms[choice_df_dms.task=='sham'])

C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\2406972439.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choice_df['dist_shifted'] = (
C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\2406972439.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choice_df['dist_shifted'] = (
C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\2406972439.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [255]:
# choose a condition
cond = 'dms'

# df based on condition
cond_df = {'dms': final_df_dms,
          'dls': final_df_dls,
          'sham': final_df_sham}[cond]

pre_rew_an_dict, pre_unrew_an_dict, pre_diff_dict = {}, {}, {}
post_rew_an_dict, post_unrew_an_dict, post_diff_dict = {}, {}, {}

# Add NaNs in the middle
nan_series = pd.Series([np.nan], index=[0])
for an in cond_df.animal.unique():
    # calc for each animal, for each rp
    for rewprob in [10, 15, 46, 80]:
        unrew = (final_df[(final_df['rewprob'] == rewprob) & (final_df['reward'] == 0) & (final_df['animal']==an)]
            .set_index('offset')['dist_shifted'])
        rew = (final_df[(final_df['rewprob'] == rewprob) & (final_df['reward'] == 1) & (final_df['animal']==an)]
            .set_index('offset')['dist_shifted'])
    
        pre_rew_an_dict[(an, rewprob)] = pd.concat([rew.iloc[:20], nan_series, rew.iloc[20:]])
        pre_unrew_an_dict[(an, rewprob)] = pd.concat([unrew.iloc[:20], nan_series, unrew.iloc[20:]])
        pre_diff_dict[(an, rewprob)] = pre_unrew_an_dict[(an, rewprob)] - pre_rew_an_dict[(an, rewprob)]

        unrew = (cond_df[(cond_df['rewprob'] == rewprob) & (cond_df['reward'] == 0) & (cond_df['animal']==an)]
            .set_index('offset')['dist_shifted'])
        rew = (cond_df[(cond_df['rewprob'] == rewprob) & (cond_df['reward'] == 1) & (cond_df['animal']==an)]
            .set_index('offset')['dist_shifted'])
    
        post_rew_an_dict[(an, rewprob)] = pd.concat([rew.iloc[:20], nan_series, rew.iloc[20:]])
        post_unrew_an_dict[(an, rewprob)] = pd.concat([unrew.iloc[:20], nan_series, unrew.iloc[20:]])
        post_diff_dict[(an, rewprob)] = post_unrew_an_dict[(an, rewprob)] - post_rew_an_dict[(an, rewprob)]

colors = sns.color_palette("crest", as_cmap=True)(np.linspace(0, 1, 4))
pre_mean = pd.DataFrame(pre_diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)].mean().to_numpy()
pre_sem = pd.DataFrame(pre_diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)].sem().to_numpy()
mean_ = pd.DataFrame(post_diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)].mean().to_numpy()
sem_ = pd.DataFrame(post_diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)].sem().to_numpy()
# plt.plot(np.arange(-hist+1, hist), mea0-mea1, 'k', label = 'Original')
# plt.plot(np.arange(-hist+1, hist), temp/len(mat_dict), 'xkcd:coral', label = 'PS matched')

# plt.fill_between(np.arange(-hist+1, hist), (temp/len(mat_dict))-sem(np.array(list(mat_dict.values()))),
#                  (temp/len(mat_dict))+sem(np.array(list(mat_dict.values()))),color = 'xkcd:coral', alpha = 0.2)
fig = plt.figure(figsize=(15, 4))
for ind, rewprob in enumerate([10, 15, 46, 80]):
    ax = plt.subplot(1, 4, ind+1)
    ax.plot(np.arange(-hist+1, hist), pre_mean[ind], '-', color = colors[ind], label = rewprob)
    ax.fill_between(
        np.arange(-hist+1, hist), pre_mean[ind] - pre_sem[ind], pre_mean[ind] + pre_sem[ind],
        color=colors[ind], alpha=0.6
    )
    ax.plot(np.arange(-hist+1, hist), mean_[ind], '-', color = colors[ind], label = rewprob, alpha = 0.5)
    ax.fill_between(
        np.arange(-hist+1, hist), mean_[ind] - sem_[ind], mean_[ind] + sem_[ind],
        color=colors[ind], alpha=0.2
    )
    ax.axvline(0, color = 'k', linestyle = '--', alpha = 0.5)
    ax.set_ylim(-0.15, 0.3)
    ax.set_title(f'{rewprob}% reward', fontsize = 'large')
    plt.yticks(color = 'grey', fontsize = 'large')
    plt.xticks(color = 'grey', fontsize = 'large')
    


# fig.supylabel('$\Delta$ distance due to single trial outcome, \n No reward $\minus$ Reward', fontsize = 'x-large')
plt.suptitle(f'{cond}, n = {len(cond_df.animal.unique())}', fontsize = 'x-large')
# plt.xlabel('Trial history', fontsize = 'x-large')
# plt.legend()
# plt.ylim(0, 0.08)
# plt.yticks([0.00, 0.02, 0.04, 0.06, 0.08], [0.00, '', '', '', 0.08], fontsize = 'large', color = 'grey')

sns.despine()
plt.tight_layout()

# Variability from start to end of training

In [267]:
fig = plt.figure(figsize = (15, 6))
# sessdf['entropy'] = (sessdf.groupby(['animal', 'session'], as_index = False)
#                      .port
#                      .rolling(5, center=True)
#                      .apply(lambda x: entropy(calc_prob(x), base = 2))
#                      .port)
for i in range(1, 9):
    final_df = calc_shifted_entropy(sessdf[sessdf.sess_bin == i])
    rew_an_dict, unrew_an_dict, diff_dict = {}, {}, {}
    # Add NaNs in the middle
    nan_series = pd.Series([np.nan] * 5, index=range(-2, 3))

    for an in final_df.animal.unique():
        # calc for each animal, for each rp
        for rewprob in [10, 15, 46, 80]:
            unrew = (final_df[(final_df['rewprob'] == rewprob) & (final_df['reward'] == 0) & (final_df['animal']==an)]
                .set_index('offset')['entropy_shifted'])
            rew = (final_df[(final_df['rewprob'] == rewprob) & (final_df['reward'] == 1) & (final_df['animal']==an)]
                .set_index('offset')['entropy_shifted'])
        
            rew_an_dict[(an, rewprob)] = pd.concat([rew.iloc[:18], nan_series, rew.iloc[18:]])
            unrew_an_dict[(an, rewprob)] = pd.concat([unrew.iloc[:18], nan_series, unrew.iloc[18:]])
            diff_dict[(an, rewprob)] = unrew_an_dict[(an, rewprob)] - rew_an_dict[(an, rewprob)]
        # if diff_dict[(an, rewprob)].isna().sum() > 0:
        #     print(an, rewprob)
    # data = pd.DataFrame(diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)]
    colors = sns.color_palette("crest", as_cmap=True)(np.linspace(0, 1, 4))
    mean_ = pd.DataFrame(diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)].mean().to_numpy()
    sem_ = pd.DataFrame(diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)].sem().to_numpy()
    # plt.plot(np.arange(-hist+1, hist), mea0-mea1, 'k', label = 'Original')
    # plt.plot(np.arange(-hist+1, hist), temp/len(mat_dict), 'xkcd:coral', label = 'PS matched')

    # plt.fill_between(np.arange(-hist+1, hist), (temp/len(mat_dict))-sem(np.array(list(mat_dict.values()))),
    #                  (temp/len(mat_dict))+sem(np.array(list(mat_dict.values()))),color = 'xkcd:coral', alpha = 0.2)
    ax = plt.subplot(2, 4, i)
    ax.axvline(0, color = 'k', linestyle = '--', alpha = 0.5)
    for ind, rewprob in enumerate([10, 15, 46, 80]):
        ax.plot(np.arange(-hist+1, hist), mean_[ind], '-', color = colors[ind], label = rewprob)
        ax.fill_between(
            np.arange(-hist+1, hist), mean_[ind] - sem_[ind], mean_[ind] + sem_[ind],
            color=colors[ind], alpha=0.2
        )
    ax.set_title(f'sess bin {i}', fontsize = 'large')
    ax.set_ylim(-0.2, 0.2)

    # plt.axvline(0, color = 'k', linestyle = '--')
    # plt.ylabel('$\Delta$ entropy due to single trial outcome, \n No reward $\minus$ Reward', fontsize = 'x-large')
    # plt.title('All task structures')
    # plt.xlabel('Trial history', fontsize = 'x-large')
    # plt.legend()
    # plt.ylim(0, 0.08)
    # plt.yticks([0.00, 0.02, 0.04, 0.06, 0.08], [0.00, '', '', '', 0.08], fontsize = 'large', color = 'grey')
    plt.yticks(color = 'grey', fontsize = 'large')
    plt.xticks(color = 'grey', fontsize = 'large')
    sns.despine()
    plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\3113863270.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choice_df['entropy_shifted'] = (
C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\3113863270.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choice_df['entropy_shifted'] = (
C:\Users\dlab\AppData\Local\Temp\ipykernel_87020\3113863270.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 